In [1]:
#import packages
import datetime as dt 
import lxml
from lxml import html, etree
import requests
import numpy as np
import pandas as pd
import pandas_datareader
from pandas_datareader import data as pdr
import time
from time import sleep
import string
import yfinance as yf
from bs4 import BeautifulSoup
import urllib
import io
import os
import pandasql
from pandasql import sqldf
import re
import gc
import zipfile
from zipfile import ZipFile
from io import StringIO
import sys

from os import listdir
from os.path import isfile, join
import psutil

from collections import Counter
import linecache
import tracemalloc

import reshape


c:\users\daniel\appdata\local\programs\python\python38-32\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
os.getcwd()

'D:\\GitHub\\finance'

In [3]:
def display_top(snapshot, key_type='lineno', limit=3):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        # replace "/path/to/module/file.py" with "module/file.py"
        filename = os.sep.join(frame.filename.split(os.sep)[-2:])
        print("#%s: %s:%s: %.1f KiB"
              % (index, filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [4]:
#get ticker, company name and CIK numbers 
ticker_url = "https://www.sec.gov/include/ticker.txt"
ticker_request = requests.get(ticker_url).content
ticker_df = pd.read_csv(io.StringIO(ticker_request.decode('utf-8')),sep="\t")
ticker_df.columns = ['ticker', 'cik']
ticker_df["ticker"] = ticker_df["ticker"].str.lower()


#get sp500 tickers 
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
sp500 = pd.DataFrame(sp500[0]['Symbol'])
sp500.columns = ['ticker']
sp500["ticker"] = sp500["ticker"].str.lower()
#print(sp500)

#get cik of sp 500
sample = pd.merge(ticker_df,sp500,on='ticker')
print(sample )

    ticker      cik
0      aal     6201
1      aap  1158449
2     aapl   320193
3     abbv  1551152
4      abc  1140859
..     ...      ...
495    yum  1041061
496    zbh  1136869
497   zbra   877212
498   zion   109380
499    zts  1555280

[500 rows x 2 columns]


In [5]:
#get tickers in the sp 500 and get easy stats on them
#get avg price by year, and current price of stock 
def get_easy_stats(df):
    
    easy_df = []
    
    start_dt = dt.datetime(2013,1,1)
    end_dt = dt.datetime(2019,12,31)
    
    for index, row in df.iterrows():
        
        try: 
        
            sleep(0.2)
            #data_ts = pdr.get_data_yahoo(row['ticker'], start_dt, end_dt)
            data_ts = yf.download(row['ticker'], start= start_dt, end= end_dt, progress=False)
            #print(data_ts)
                                      
            data_ts['year'] = data_ts.index.year
            #get average price per share 
            agg_df = data_ts.groupby('year', as_index=False)['Adj Close'].mean()
            agg_df['ticker'] = row['ticker']
            agg_df.columns = ['year','avg_price', 'ticker']
            agg_df['year']=  agg_df['year'].astype(str)
    #       print(agg_df)
            current_price = pdr.get_data_yahoo(row['ticker']).last('1D')['Adj Close'][0].astype(int)

            #turn agg_df wide  for price rows
            wide_df = agg_df.pivot(index='ticker', columns='year', values=['avg_price'])

            wide_df.columns = list(map("".join, wide_df.columns))
            wide_df['current_price'] = current_price
            print(wide_df)

            easy_df.append(wide_df)
            del agg_df
            del wide_df
        
        except Exception as ex:
            print(row['ticker'] + " failed")
            print(ex)
            
    
    easy_df = pd.concat(easy_df)
    #easy_df.to_csv("easy_stats_sp500.csv")
    return(easy_df)
#function end

start = time.time()
#run to generate easy stat csv
easy_stat_df = get_easy_stats(sample)
easy_stat_df.to_csv("easy_stats_sp500.csv")
print(easy_stat_df)

end = time.time()
print("finished getting easy stats for sp500")
print(end - start)



In [6]:
#read in the easy stats of sp500
#easy_stat_df.to_csv("easy_stats_sp500.csv")
easy_stat_df = pd.read_csv("easy_stats_sp500.csv")
print(easy_stat_df)

    ticker  avg_price2013  avg_price2014  avg_price2015  avg_price2016  \
0      aal      17.146522      36.755626      42.940213      36.716199   
1      aap      83.696200     128.910822     161.370373     153.556964   
2     aapl      59.771992      83.652771     110.696663      98.462125   
3     abbv      32.361064      42.512583      49.072086      49.757348   
4      abc      50.829051      67.690750      95.513013      77.419414   
..     ...            ...            ...            ...            ...   
465    yum      43.882041      47.608164      52.553995      55.586190   
466    zbh      75.714952      96.441867     104.284561     109.899370   
467   zbra      46.445833      71.196667      89.878492      64.032024   
468   zion      24.429700      26.550984      26.060497      26.514911   
469    zts      30.203858      32.603061      44.496182      46.556359   

     avg_price2017  avg_price2018  avg_price2019  current_price  
0        46.100999      41.898407      30.579

In [7]:
#set up script to traverse directory of quarterly submissions
def generate_finance_table_csv():

    #store all finances in a table
    finance_list = []

    #go into directory with data
    file_os = 'D:\\Finance Data'
    
    
    code_os = 'D:\\GitHub\\finance'
    start = time.time()
    
    #go into file directory
    os.chdir(file_os)
    
    for i in os.listdir():
        file_path = i
        #if its a quarterly filing data, parse it
        if 'q1' in file_path or 'q2' in file_path or 'q3' in file_path or 'q4' in file_path:
            try:
                tracemalloc.start()
                print("starting: " + file_path)

                sub = pd.read_csv(file_path + '/' + 'sub.txt', sep = '\t', encoding = "ISO-8859-1", iterator = True, chunksize =100000)
                sub = pd.concat(sub)
                #clean up form column and filter to just 10-K submissions
                sub['form'] = sub['form'].replace(np.nan, '', regex=True)
                sub = sub[sub['form'] == '10-K']
                #filter to the s&p500
                sub = pd.merge(sub,sample, on = ['cik'])

                pre = pd.read_csv(file_path + '/' + 'pre.txt', sep = '\t', encoding = "ISO-8859-1", chunksize =1000000) 
                num = pd.read_csv(file_path + '/' + 'num.txt', sep = '\t', encoding = "ISO-8859-1",  chunksize =100000)

                print("joining num to sub")
                num_sub = pd.DataFrame()              
                #join in numbers to submissions in chunks
                for chunks in num: 
                    #print(chunks)
                    num_sub = pd.concat([num_sub, sub.merge(chunks, on=['adsh'])])

                #no longer need sub
                del sub

                print("joining pre to numsub")
                nsp = pd.DataFrame()
                #add in plabel and stmt info
                for chunks in pre:
                    #print(chunks)
                    nsp = pd.concat([nsp, num_sub.merge(chunks, on=['adsh', 'tag', 'version'])])
                    nsp = nsp[nsp['stmt'].isin(['BS', 'IS', 'CF', 'CI', 'EQ'])]
                # delete files no longer needed
                del num_sub
                del pre
                del num


                #print(nsp.columns)

                nsp = nsp[['name', 'sic', 'fye', 'form', 'period', 
                           'fy', 'fp', 'filed','ticker', 'cik','ddate', 
                           'qtrs', 'uom', 'value', 'adsh','stmt',  'tag', 
                           'version', 'plabel']]

                #add finances to master table
                finance_list.append(nsp)
                del nsp
                print("processed_data: " + file_path)
                gc.collect()
                snapshot = tracemalloc.take_snapshot()
                display_top(snapshot)
            except Exception as ex:
                print("failed: " + file_path)
                print(str(ex))
    
    #switch back to github directory 
    os.chdir(code_os)
    
    
    finance_table = pd.concat(finance_list)
    del finance_list 
    finance_table.to_csv("total_finance_table.csv")
    end = time.time()
    print("finished traversing files in X secs")
    print(end - start)
    #return(finance_table)

In [8]:
# #run function to generate finance table csv
generate_finance_table_csv()

In [12]:
#main function goes through and tries to parse out the historical data we need: VERY LONG 
def parse_data(df):
    gc.collect()
    ############################################################
    #process data a bit 
    
    #lower case plabel 
    df['plabel'] = df['plabel'].str.lower()
    
    #turn things into an integer
    df['qtrs'] = df.qtrs.astype(int)
    df['period'] = df.period.astype(int)
    df['fy'] = df.fy.astype(int)
    df['fye'] = df.fye.astype(int)
    df['ddate'] = df.ddate.astype(int)
    df['ddate_prev'] = df.ddate.astype(int) + 10000
    
    #turn things to strings
    df['period'] = df['period'].astype(str)
    df['fye'] = df['fye'].astype(str)
    df['fy'] = df['fy'].astype(str)
    df['ddate'] = df['ddate'].astype(str)
    df['ddate_prev'] = df['ddate_prev'].astype(str)
    #pad fye with a leading 0 
    df['fye'] = df['fye'].apply(lambda x: x.zfill(4))
    
    #fill nas inplace = true changes the df directly
    df['period'].fillna('', inplace=True)
    df['fy'].fillna('', inplace=True)
    df['fye'].fillna('', inplace=True)
    df['plabel'].fillna('', inplace=True)
    df['ddate'].fillna('', inplace=True)
    df['ddate_prev'].fillna('', inplace=True)
    
    #create a joinable key
    df['id'] = df['fy'].astype(str) + '_' + df['cik'].astype(str)

    #get distinct company and years
    key_df = pd.DataFrame(df.id.unique(), columns = ['id'])
    gc.collect()
    
    #######################################################################################################
    #define tags
    
    #for shares outstanding calc (all qtrs = 4, income stmt)
    shares_outstanding_tags = ['WeightedAverageNumberOfDilutedSharesOutstanding']
    net_income_tags = ['NetIncomeLoss']
    eps_tags = ['EarningsPerShareDiluted']
    
    # for fcfe calculation  (cash flow stmt)
    cfo_tags = ['NetCashProvidedByUsedInOperatingActivities']
    debt_repayments_tags= ['RepaymentsOfDebt']
    debt_proceeds_tags = ['ProceedsFromIssuanceOfDebt']
    
    #for capital expenditure calculations (cash flow statement)
    capex_tags = ['PaymentsToAcquireProductiveAssets']
    ppe_purchase_tags = ['PaymentsToAcquirePropertyPlantAndEquipment']
    ppe_sale_tags = ['ProceedsFromSaleOfPropertyPlantAndEquipment']
    #qtrs = 0; for ppe on balance sheet
    ppe_tags = ['PropertyPlantAndEquipmentNet']

    #for ebitda calculation
    revenue_tags = ['Revenues']
    operating_income_tags = ['OperatingIncomeLoss']
    dep_amort_tags = ['DepreciationDepletionAndAmortization']
    
    #######################################################################################################
    #pull rows
    
#shares outstanding
    shares_outstanding_rows = df[
        (df['tag'].isin(shares_outstanding_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.uom.str.contains('shares',na=False))
    ]

    shares_outstanding_agg = shares_outstanding_rows.groupby(['id'], as_index=False)['value'].max()  
    shares_outstanding_agg.columns = ['id', 'shares_outstanding']
    del shares_outstanding_rows
#net income
    net_income_rows = df[
        (df['tag'].isin(net_income_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    net_income_agg = net_income_rows.groupby(['id'], as_index=False)['value'].max()  
    net_income_agg.columns = ['id', 'net_income']
    del net_income_rows
#eps
    eps_rows = df[
        (df['tag'].isin(eps_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    eps_agg = eps_rows.groupby(['id'], as_index=False)['value'].max()  
    eps_agg.columns = ['id', 'eps']
    del eps_rows
#cash flow from operations    
    cfo_rows = df[
        (df['tag'].isin(cfo_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    cfo_agg = cfo_rows.groupby(['id'], as_index=False)['value'].max()  
    cfo_agg.columns = ['id', 'cfo']
    del cfo_rows
#debt repayment 
    debt_repayments_rows = df[
        (df['tag'].isin(debt_repayments_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    debt_repayments_agg = debt_repayments_rows.groupby(['id'], as_index=False)['value'].max()  
    debt_repayments_agg.columns = ['id', 'debt_repayments']
    del debt_repayments_rows
#debt proceeds 
    debt_proceeds_rows = df[
        (df['tag'].isin(debt_proceeds_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    debt_proceeds_agg = debt_proceeds_rows.groupby(['id'], as_index=False)['value'].max()  
    debt_proceeds_agg.columns = ['id', 'debt_proceeds']
    del debt_proceeds_rows
#cap ex
    capex_rows = df[
        (df['tag'].isin(capex_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    capex_agg = capex_rows.groupby(['id'], as_index=False)['value'].max()  
    capex_agg.columns = ['id', 'capex']
    del capex_rows
#ppe purchase
    ppe_purchase_rows = df[
        (df['tag'].isin(ppe_purchase_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    ppe_purchase_agg = ppe_purchase_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_purchase_agg.columns = ['id', 'ppe_purchase']
    del ppe_purchase_rows
#ppe sale 
    ppe_sale_rows = df[
        (df['tag'].isin(ppe_sale_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    ppe_sale_agg = ppe_sale_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_sale_agg.columns = ['id', 'ppe_sale']
    del ppe_sale_rows
#ppe , qtrs = 0; for ppe on balance sheet
    ppe_rows = df[
        (df['tag'].isin(ppe_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 0)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    ppe_agg = ppe_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_agg.columns = ['id', 'ppe']
    del ppe_rows
#get PPE of the year before
    ppe_prev_rows = df[
        (df['tag'].isin(ppe_tags))
        & (df['ddate_prev'] == (df['fy'] + df['fye']))
        & (df.qtrs == 0)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    ppe_prev_agg = ppe_prev_rows.groupby(['id'], as_index=False)['value'].max()  
    ppe_prev_agg.columns = ['id', 'ppe_prev']
    del ppe_prev_rows
#revenue 
    revenue_rows = df[
        (df['tag'].isin(revenue_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    revenue_agg = revenue_rows.groupby(['id'], as_index=False)['value'].max()  
    revenue_agg.columns = ['id', 'revenue']
    del revenue_rows
#operating income
    operating_income_rows = df[
        (df['tag'].isin(operating_income_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    operating_income_agg = operating_income_rows.groupby(['id'], as_index=False)['value'].max()  
    operating_income_agg.columns = ['id', 'operating_income']
    del operating_income_rows
#depreciation and amortization
    dep_amort_rows = df[
        (df['tag'].isin(dep_amort_tags))
        & (df['ddate'] == (df['fy'] + df['fye']))
        & (df.qtrs == 4)
    ]
    #get max number of shares outstandiing from income statement  for each company and fiscal year
    dep_amort_agg = dep_amort_rows.groupby(['id'], as_index=False)['value'].max()  
    dep_amort_agg.columns = ['id', 'dep_amort']
    del dep_amort_rows


#     #get current assets 
#     #get current liabilites
#     #get long term debt on balance sheet
#     #get short term debt on balance sheet 
#     #get cash and cash equivalents

#     #######################################################################################################
    #add financials to key_df
    merged_df = key_df.merge(shares_outstanding_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(net_income_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(eps_agg, on = ['id'], how = "left")
    
    merged_df = merged_df.merge(cfo_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(debt_proceeds_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(debt_repayments_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(capex_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_purchase_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_sale_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(ppe_prev_agg, on = ['id'], how = "left")
    
    merged_df = merged_df.merge(revenue_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(operating_income_agg, on = ['id'], how = "left")
    merged_df = merged_df.merge(dep_amort_agg, on = ['id'], how = "left")
    
    
#     #######################################################################################################
#     #delete unneeded things 
    del shares_outstanding_agg
    del net_income_agg
    del eps_agg
    del cfo_agg
    del debt_proceeds_agg
    del debt_repayments_agg
    del capex_agg
    del ppe_purchase_agg
    del ppe_sale_agg
    del ppe_agg
    del ppe_prev_agg
    del revenue_agg
    del operating_income_agg
    del dep_amort_agg
    return(merged_df)



In [17]:
# process the data in chunks first, return ticker level dataset 
def init_process_data():
    
    processed_list = []
    
    chunksize = 10 ** 5
    for chunk in pd.read_csv("total_finance_table.csv", chunksize=chunksize):  
        #for each chunk, get finance info on a company, yearly level
        processed_list.append(parse_data(chunk))
    #turn list of dataframes into one dataframe
    processed_table = pd.concat(processed_list)
    

    #split id back into year and cik
    processed_table[['year','cik']] = processed_table.id.str.split(pat = "_", expand=True) 
    print("printing pre-aggregated data")
    print(processed_table)
    del processed_list
    #######################################################################################################
    #aggregate up across chunks 
    agg_df = sqldf(
    """
     SELECT
    *, 
    cfo - capex_coalesced + net_debt_proceeds AS fcfe, 
    cfo - capex_coalesced AS fcfe_modded
    FROM 
    (
         SELECT 
        *,
        IFNULL(operating_income,0) + IFNULL(dep_amort,0) AS ebitda, 

        CASE WHEN shares_outstanding IS NULL THEN net_income/NULLIF(eps,0) 
             ELSE shares_outstanding 
             END AS shares_outstanding_coalesced, 

        CASE WHEN capex IS NOT NULL THEN capex 
             WHEN ppe_purchase IS NOT NULL THEN ppe_purchase - IFNULL(ppe_sale,0)
             WHEN ppe IS NOT NULL THEN ppe - IFNULL(ppe_prev,0) 
             END AS capex_coalesced, 
             
        IFNULL(debt_proceeds,0) - IFNULL(debt_repayments,0) AS net_debt_proceeds
        
        FROM 
        (
            SELECT  
            year,
            CAST(cik AS INT64) as cik,
            MAX(shares_outstanding) AS shares_outstanding,
            MAX(net_income) AS net_income,
            MAX(eps) AS eps,
            MAX(cfo) AS cfo,
            MAX(debt_proceeds) AS debt_proceeds,
            MAX(debt_repayments) AS debt_repayments,
            MAX(capex) AS capex,
            MAX(ppe_purchase) AS ppe_purchase,
            MAX(ppe_sale) AS ppe_sale,
            MAX(ppe) AS ppe,
            MAX(ppe_prev) AS ppe_prev, 
            MAX(revenue) AS revenue,
            MAX(operating_income) AS operating_income,
            MAX(dep_amort) AS dep_amort
            FROM 
            processed_table 
            GROUP BY     
            year,
            cik
            ORDER BY 2 DESC
        )
        )
            
    """)
    
    del processed_table 
    print(agg_df.columns)
    
#     #######################################################################################################
    # turn long data wide 
    wide_df = agg_df.pivot(index='cik', columns='year', values=['shares_outstanding', 'net_income', 'eps', 'cfo',
       'debt_proceeds', 'debt_repayments', 'capex', 'ppe_purchase', 'ppe_sale',
       'ppe', 'ppe_prev', 'revenue', 'operating_income', 'dep_amort', 'ebitda',
       'shares_outstanding_coalesced', 'capex_coalesced', 'net_debt_proceeds',
       'fcfe', 'fcfe_modded'])
    
    del agg_df
    
#     ####################################################################################################### 
    #fix columns
    wide_df.columns = list(map("".join, wide_df.columns))
    
    #add in tickers
    wide_df = wide_df.merge(sample, on = ['cik'])
    
    #drop weird columns 
    wide_df = wide_df[wide_df.columns.drop(list(wide_df.filter(regex='1218')))]
    wide_df = wide_df[wide_df.columns.drop(list(wide_df.filter(regex='2020')))]
    
    #read in easy stats csv
    easy_stat_df = pd.read_csv("easy_stats_sp500.csv")
    
    #join in easy stats
    wide_df = wide_df.merge(easy_stat_df, on = 'ticker')
    
#     ####################################################################################################### 
#     # add dataquality columns - 
#     #WTK: how many revenue, ebitda, fcfe and shares_outstanding columns are available with 2019 info available
    
#     # WTK: calculated estimated cagrs (FCFE growth, Revenue growth) and multiples (fcfe yield, EV/EBITDA)
    
#     # forecast earnings, growth, multiple expansion or decrease over 10 years and get estimated IRR of stock
    
    
#     ####################################################################################################### 
    #write to csv
    print("printing df to write to csv")
    print(wide_df)
    wide_df.to_csv("sp500_financials.csv")
    return(wide_df)
    
start = time.time()

print("starting to process data")
init_df = init_process_data()

end = time.time()
print("finished processing data")
print(end - start)

starting to process data
printing pre-aggregated data
               id  shares_outstanding    net_income    eps           cfo  \
0       2013_4904        4.870410e+08  1.480000e+09   3.04  4.106000e+09   
1       2013_4977        4.674080e+08  3.158000e+09   6.76  1.054700e+10   
2       2013_5513                 NaN  8.581000e+08   3.23  1.031500e+09   
3       2013_6201        2.763260e+08 -1.526000e+09 -11.25  9.490000e+08   
4       2013_7084        6.630000e+08  1.342000e+09   2.02  5.226000e+09   
..            ...                 ...           ...    ...           ...   
414  2019_1688568                 NaN  1.257000e+09   4.47  1.783000e+09   
415  2019_1757898                 NaN  3.040510e+08   3.56  5.395050e+08   
416    2019_56873        8.180000e+08  3.110000e+09   3.76  4.164000e+09   
417   2019_849399        6.320000e+08  3.100000e+07   0.05  1.495000e+09   
418  2020_1613103        1.351100e+09  4.789000e+09   3.54  7.234000e+09   

     debt_proceeds  debt_repaymen

finished processing data
29.261815309524536


In [ ]:
for i in init_df.columns: print(i)
#print(init_df)

In [19]:
#assess quality of data here 
pt_good = sqldf("""SELECT DISTINCT ticker FROM init_df 
                    WHERE shares_outstanding_coalesced2019 > 0
                """)
print(pt_good)

    ticker
0      abt
1      amd
2      apd
3     swks
4      hwm
..     ...
441    dis
442   amcr
443    dow
444   ctva
445    ste

[446 rows x 1 columns]


In [ ]:
#get other ticker info from other script (current price, dividend yield, market cap, shares outsanding , sector)

In [ ]:
#get 2018 walmart ticker = wmt, fy = 2018
# analyze missing info 
dq_df = pd.read_csv("total_finance_table.csv")

In [ ]:
dq_df.columns

In [ ]:
ex = dq_df[(dq_df['ticker'] == 'dis')
           #& (dq_df['fy'] == 2015.0)
           
           ]

In [ ]:
for index, row in ex.iterrows():
    print(row)